<a href="https://colab.research.google.com/github/clarkde5/jams-pub/blob/main/colabs/jams-env-doctr-lease.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Environment**

## **Bootstrap public repo**

In [ ]:
from pathlib import Path

if not Path(f"/content").is_dir():
  %cd ~
else:
  %cd /content

if not Path(f"src/jams-pub").is_dir():
  !git clone https://github.com/clarkde5/jams-pub src/jams-pub
else:
  print("src/jams-pub already exists")

import sys
import os

if not "src/jams-pub/py-lib" in sys.path:
  sys.path.append(os.path.abspath("src/jams-pub/py-lib"))

print(sys.path)

## **Pull private resources**

In [ ]:
runOptional = True
def CallPrivateEnvSetup():
  from pathlib import Path

  home = str(Path.home())

  try:
    from google.colab import userdata
    try:
      rsa_private_key = userdata.get('PrivateKey')
    except:
      rsa_private_key = ""
      print("PrivateKey is required, but not found in secrets / colab userdata")
      return
  except:
    print("google.colab could not be loaded")
    if not Path(f"{home}/.ssh/id_rsa").is_file():
      print("rsa key does not exist, please create manually")
      return
    else:
      print("rsa key already exists")

  if not Path(f"{home}/.ssh/id_rsa").is_file():
    !python src/jams-pub/env-setup.py "{rsa_private_key}"

  if not Path(f"{home}/.ssh/id_rsa").is_file():
    print("env-setup failed to create rsa key")
    return
  else:
    #!apt-get install dos2unix -y
    #!dos2unix ~/.ssh/id_rsa
    !sed -i 's/\r$//' ~/.ssh/id_rsa

  if not Path(f"src/jams").is_dir():
    !git clone git@github.com:clarkde5/jams.git src/jams
  else:
    print("src/jams already exists")

if __name__ == "__main__":
  if runOptional == True:
    CallPrivateEnvSetup()
  else:
    print("Not running optional cell")

# **Parse Lease Output**

**NOTE:** Requires [Pull private resources](https://colab.research.google.com/github/clarkde5/jams-pub/blob/main/colabs/jams-env-doctr.ipynb#scrollTo=CyQkr3hoftql)

In [ ]:
runOptional = True
def main():
  import json
  import lease
  
  f = open('src/jams/output/Aug 23 Lease-docTR.json')
  data = json.load(f)

  contracts = []
  serialNumbers = []
  paymentDue = []

  invoice_number = getInvoiceNumber(0,data["pages"][0])

  for page_idx,page in enumerate(data["pages"]):
    contracts += getContractsForPage(page_idx,page)
    serialNumbers += getSerialNumbersForPage(page_idx,page)
    paymentDue += getPaymentDue(page_idx,page)

  contractSerials = contracts + serialNumbers + paymentDue
  contractSerials = sorted(contractSerials, key = lambda x: x["pdf_y"])

  response_list = {"pages" : convertToJson(contractSerials), "invoice_number": invoice_number}
  response_list = calculateTotals(response_list)


  json_formatted_str = json.dumps(response_list, indent=2)
  print(json_formatted_str)

if __name__ == "__main__":
  if runOptional == True:
    main()
  else:
    print("Not running optional cell")